In [1]:
import pandas as pd
import spacy
import stanza
from stanza.utils.conll import CoNLL
import os
import re

def callFreeling(text :str, splitSentences = True) -> list[list[dict]]:
    open(os.path.abspath("")+"/tmp.txt", "w+").write(text)
    os.system(f"bash ./callFreeling.sh {os.path.abspath("")+"/tmp.txt"} {os.path.abspath("")+"/tmp2.txt"}")
    out = open(os.path.abspath("")+"/tmp2.txt", "r").read().split("\n\n") if splitSentences else [open(os.path.abspath("")+"/tmp2.txt", "r").read().replace("\n\n", "\n")]
    os.remove(os.path.abspath("")+"/tmp.txt")
    os.remove(os.path.abspath("")+"/tmp2.txt")
    out = [i.split("\n") for i in out]
    columns = "ID FORM LEMMA TAG SHORT_TAG MSD NEC SENSE SYNTAX DEPHEAD DEPREL COREF SRL".split(" ")
    out = [[{columns[i] : re.split(" +", word)[i] for i in range(len(re.split(" +", word)))} for word in sent]for sent in out]
    return out

In [2]:
nlp_stanza = stanza.Pipeline(lang='es', processors='tokenize,ner,mwt,pos,lemma,depparse')
nlp_spacy = spacy.load("es_dep_news_trf")

2025-03-23 20:49:51 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-03-23 20:49:52 INFO: Downloaded file to /home/ivan/stanza_resources/resources.json
2025-03-23 20:49:53 INFO: Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |
| ner       | conll02           |

2025-03-23 20:49:53 INFO: Using device: cpu
2025-03-23 20:49:53 INFO: Loading: tokenize
2025-03-23 20:49:54 INFO: Loading: mwt
2025-03-23 20:49:54 INFO: Loading: pos
2025-03-23 20:49:56 INFO: Loading: lemma
2025-03-23 20:49:57 INFO: Loading: depparse
2025-03-23 20:49:58 INFO: Loading: ner
2025-03-23 20:50:00 INFO: Done loading processors!
/home/ivan/miniconda3/lib/python3.12/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'es_dep_news_trf' (3.7.2) was trained with spaCy v3.7.0 and may not be 100% compatible with the current version (3.8.2). If you

In [3]:
baselineFilePath = "/home/ivan/Desktop/Uni/TFG/RepositorioTFG/Datasets/UD_Spanish-AnCora/es_ancora-ud-train.conllu" #Baseline corpus path. It's not included in the repo, change this at your own will. Must be in conllu format
baseline = CoNLL.conll2doc(baselineFilePath)
sentences = [i.text for i in baseline.sentences]

In [ ]:
# SPACY MODEL
spacy_output = nlp_spacy(" ".join(sentences))

# STANZA MODEL
stanza_output = nlp_stanza(" ".join(sentences))

In [ ]:
# NOTE: Execute this only if you have installed Freeling!!
# FREELING MODEL
freeling_parser = callFreeling(" ".join(sentences))

output saved in /home/ivan/Desktop/Uni/TFG/RepositorioTFG/freelingOutput.conll


In [5]:
# NOTE: Execute this only if you have installed Malt Parser!!
# NOTE: Also check that the Malt Parser folder is in the same directory as the callMaltParser.sh script
# MALT PARSER: chances are this will be removed, as I can't get it to work properly and it depends on freeling sentencizer and tagger
if baselineFilePath[len(baselineFilePath)-6:] != ".conll":
    os.system(f"python conllu-to-conll-standard.py {baselineFilePath} {os.path.abspath("")+"/corpus.conll"}")
    os.system(f"bash callMaltParser.sh {os.path.abspath("")+"/corpus.conll"}")
else:
    os.system(f"bash callMaltParser.sh {baselineFilePath}")
malt_parser = CoNLL.conll2doc(os.path.abspath("")+"/maltParserOut.conll")

/home/ivan/Desktop/Uni/TFG/RepositorioTFG/corpus.conll


-----------------------------------------------------------------------------
                          MaltParser 1.9.2                             
-----------------------------------------------------------------------------
         MALT (Models and Algorithms for Language Technology) Group          
             Vaxjo University and Uppsala University                         
                             Sweden                                          
-----------------------------------------------------------------------------

Started: Wed Mar 05 17:46:57 CET 2025
  Transition system    : Non-Projective
  Parser configuration : Covington with allow_root=true and allow_shift=false
  Data Format          : /espmalt-1.0/conllx.xml
.          	      1	      2s	    145MB
.          	     10	      2s	    145MB
.          	    100	      3s	    190MB
..........	   1000	     12s	    365MB
.......    	   1721	     15s	    304MB
Parsing time: 00:00:14 (14550 ms)
Finished: Wed Mar 05 17:47

output saved in /home/ivan/Desktop/Uni/TFG/RepositorioTFG/maltParserOut.conll


In [6]:
print("Baseline: ", len(list(baseline.sentences)), "sentences")
#print("Malt Parser: ", len(list(malt_parser.sentences)), "sentences")
print(f"Spacy: {len(list(spacy_output.sents))} sentences. Baseline difference: ", (len(list(spacy_output.sents))/len(list(baseline.sentences))-1)*100, "%")
print(f"Stanza: {len(list(stanza_output.sentences))} sentences. Baseline difference: ", (len(list(stanza_output.sentences))/len(list(baseline.sentences))-1)*100, "%")
print(f"freeling: {len(freeling_parser)} sentences. Baseline difference: ", (len(freeling_parser)/len(list(baseline.sentences))-1)*100, "%")

Baseline:  1721 sentences
Spacy: 1714 sentences. Baseline difference:  -0.40674026728646506 %
Stanza: 1724 sentences. Baseline difference:  0.17431725740848503 %
freeling: 1495 sentences. Baseline difference:  -13.13190005810575 %


Sentence split is inconsistent between stanza, spacy and especially freeling when compared to the baseline, which is a real problem. This highlights the difficulty of comparing syntactic parsers, as there are many more systems that play a role such as PoS tagger and sentencizer. A different approach is required, maybe disabling the sentencizer and processing the corpus sentence by sentence?

In [4]:
#Trying only dependence parsing

# Loading stanza model with sentence split disabled
nlp_stanza = stanza.Pipeline(lang='es', processors='tokenize,ner,mwt,pos,lemma,depparse', tokenize_no_ssplit=True)

# Spacy sentencizer apparently can't be disabled so each sentence in the corpus is processed individually instead. Note that this may risk sentence segmenter interference
nlp_spacy = spacy.load("es_dep_news_trf")

2025-03-08 12:38:01 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-03-08 12:38:01 INFO: Downloaded file to /home/ivan/stanza_resources/resources.json
2025-03-08 12:38:03 INFO: Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |
| ner       | conll02           |

2025-03-08 12:38:03 INFO: Using device: cpu
2025-03-08 12:38:03 INFO: Loading: tokenize
2025-03-08 12:38:03 INFO: Loading: mwt
2025-03-08 12:38:03 INFO: Loading: pos
2025-03-08 12:38:05 INFO: Loading: lemma
2025-03-08 12:38:06 INFO: Loading: depparse
2025-03-08 12:38:07 INFO: Loading: ner
2025-03-08 12:38:09 INFO: Done loading processors!


## Stanza

In [4]:
stanza_output = nlp_stanza("\n\n".join(sentences))
tag_dict = {} # Dictionary with a list of length 2 inside. Index 0 for count; index 1 for correct tag in model
for i in range(len(list(stanza_output.sentences))):
    if baseline.sentences[i].text != stanza_output.sentences[i].text:
        raise KeyError("Sentences are not equal")
    for j in range(len(list(baseline.sentences[i].words))):
        try:
            if baseline.sentences[i].words[j].deprel not in tag_dict.keys():
                tag_dict.update({baseline.sentences[i].words[j].deprel : [1,0]}) 
            else: tag_dict[baseline.sentences[i].words[j].deprel][0] += 1 
            
            if baseline.sentences[i].words[j].deprel == stanza_output.sentences[i].words[j].deprel:
                tag_dict[baseline.sentences[i].words[j].deprel][1] += 1 
        except IndexError:
            if baseline.sentences[i].words[j].deprel not in tag_dict.keys():
                tag_dict.update({baseline.sentences[i].words[j].deprel : [1,0]})
            else: tag_dict[baseline.sentences[i].words[j].deprel][0] += 1 

KeyboardInterrupt: 

In [ ]:
accuracy = {k:tag_dict[k][1]/tag_dict[k][0] for k in tag_dict.keys()}
print(f"""Accuracy of Stanza in Spanish subjects, objects and roots:
\tnominal subjects: {accuracy["nsubj"]}
\tclause subjects: {accuracy["csubj"]}
\tobjects: {accuracy["obj"]}
\troot of sentence: {accuracy["root"]}
""")

Accuracy of Stanza in Spanish subjects, objects and roots:
	nominal subjects: 0.9361924686192469
	clause subjects: 0.6595744680851063
	objects: 0.8522920203735145
	root of sentence: 0.9517722254503196



## Spacy

In [ ]:
tag_dict = {}
neqSents = 0
for i in range(len(baseline.sentences)):
    spacy_output = nlp_spacy(baseline.sentences[i].text.replace(" del ", " de el "))
    if baseline.sentences[i].text != list(spacy_output.sents)[0].text:
         neqSents+=1
    for j in range(len(baseline.sentences[i].words)):
        if baseline.sentences[i].words[j].deprel not in tag_dict.keys():
                tag_dict.update({baseline.sentences[i].words[j].deprel : [1,0]})
        else: tag_dict[baseline.sentences[i].words[j].deprel][0] += 1 
        if j < len(list(spacy_output.sents)[0]):
            """print(f"{baseline.sentences[i].words[j].text} / {list(spacy_output.sents)[0][j+offset].text} with deprel {baseline.sentences[i].words[j].deprel} / {list(spacy_output.sents)[0][j+offset].dep_.lower()}")""" # This is for debugging purposes
            if baseline.sentences[i].words[j].deprel == list(spacy_output.sents)[0][j].dep_.lower():
                tag_dict[baseline.sentences[i].words[j].deprel][1] += 1


/home/ivan/miniconda3/lib/python3.12/site-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


In [ ]:
accuracy = {k:tag_dict[k][1]/tag_dict[k][0] for k in tag_dict.keys()}
print(f"""Accuracy of Spacy in Spanish subjects, objects and roots:
\tSubjects: {accuracy["nsubj"]}
\tObjects: {accuracy["obj"]}
\tRoot of sentence: {accuracy["root"]}
""")

print(f"{neqSents} sentences were not equal, maybe due to a segmentation problem.")

Accuracy of Spacy in Spanish subjects, objects and roots:
	Subjects: 0.8657601115760112
	Objects: 0.7461799660441426
	Root of sentence: 0.9151656013945381

484 sentences were not equal, maybe due to a segmentation problem.
